In [1]:
import pandas as pd

In [5]:
df_cloroquina = pd.read_excel("ep2-cloroquina-treino.xlsx",sheet_name='train',usecols=['texto','posicao'])
x = df_cloroquina.texto
y = df_cloroquina.posicao

0           for
1       against
2       against
3       against
4       against
         ...   
5488    against
5489    against
5490        for
5491        for
5492        for
Name: posicao, Length: 5493, dtype: object
